# Codificación de posiciones de palabras
 
Convertir  los  ID  de  token  en  una  representación  vectorial  continua,  las  llamadas  incrustaciones  de  token.  En  principio,  esta  es  una  entrada  adecuada  para  un  LLM.  Sin  embargo,  una  pequeña  desventaja  de  los  LLM  es  que  su  mecanismo  de  autoatención,  no  tiene  noción  de  posición  ni  orden  para  los  tokens  dentro  de  una  secuencia.

El  mismo  ID  de  token  siempre  se  asigna  a  la  misma  representación  vectorial,  independientemente  de  dónde  se  posicione  el  ID  de  token  en  la  secuencia  de  entrada.

![Texto alternativo](./imgs/2.15.png)

En  principio,  la  incrustación  determinista  e  independiente  de  la  posición  del  ID  del  token  es  beneficiosa  para  la  reproducibilidad.  Sin  embargo,  dado  que  el  mecanismo  de  autoatención  de  los  LLM  también  es  independiente  de  la  posición,  resulta  útil  inyectar  información  de  posición  adicional  en  el  LLM.
Las  incrustaciones  posicionales  absolutas  se  asocian  directamente  con  posiciones  específicas  en  una  secuencia.  Para  cada  posición  en  la  secuencia  de  entrada,  se  añade  una  incrustación  única  a  la  incrustación  del  token  para  indicar  su  ubicación  exacta.  Por  ejemplo,  el  primer  token  tendrá  una  incrustación  posicional  específica,  el  segundo  token  otra  incrustación  distinta,  y  así  sucesivamente.

![Texto alternativo](./imgs/2.16.png)

Las incrustaciones posicionales pueden ser absolutas o relativas: las absolutas indican la posición exacta de cada token, mientras que las relativas se enfocan en la distancia entre ellos, lo que favorece la generalización a secuencias de distinta longitud. OpenAI usa incrustaciones absolutas optimizadas durante el entrenamiento, a diferencia de las fijas del Transformer original. Se crean incrustaciones iniciales con vectores de 256 dimensiones (menores que las 12 288 de GPT-3) a partir de tokens generados con el tokenizador BPE de 50 257 elementos.

In [ ]:
import torch

vocab_tam = 50257
salida_dim = 256
embedding_layer = torch.nn.Embedding(vocab_tam, salida_dim)


Si  tomamos  muestras  de  datos  del  cargador  de  datos, Incruste  cada  token  de  cada  lote  en  un  vector  de  256  dimensiones.
Si  tenemos  un  tamaño  de  lote  de  8 con  cuatro  tokens  cada  uno,  el  resultado  será  un  tensor  de  8  x  4  x  256.

In [2]:
from torch.utils.data import Dataset, DataLoader
import torch
import tiktoken

with open("../txt/The_Verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_lenght, stride):
        self.input_ids = []
        self.labels_ids = []

        tokens_ids = tokenizer.encode(txt) #Tokenzar el texto completo

        for i in range(0, len(tokens_ids) - max_lenght, stride): #Uso de una  ventana  deslizante  para  dividir  el  libro  en  secuencias  superpuestas  de  longitud  máxima
            self.input_ids.append(torch.tensor(tokens_ids[i:i+max_lenght]))
            self.labels_ids.append(torch.tensor(tokens_ids[i+1:i+max_lenght+1]))

    def __len__(self): #Devuelve  el  número  total  de  filas  en  el  conjunto  de  datos
        return len(self.labels_ids)
    
    def __getitem__(self, index): #Devuelve  una  sola  fila  del  conjunto  de  datos
        return self.input_ids[index], self.labels_ids[index]
def create_dataloader_v1(txt, batch_size=4, max_length=256,
       stride=128, shuffle=True, drop_last=True, num_workers=0):
   tokenizer = tiktoken.get_encoding("gpt2")         #inicializar toenizer          
   dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)    #Crear dataset
   dataloader = DataLoader( 
       dataset,
       batch_size=batch_size,
       shuffle=shuffle,
       drop_last=drop_last,                                  
       num_workers=num_workers                                   
   )
   return dataloader

In [4]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length, stride=max_length, shuffle=False)
data_iter = iter(dataloader)
inputs, labels = next(data_iter)
print("Ids tokens: ", inputs)
print("Forma entrada: ", inputs.shape)

Ids tokens:  tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
Forma entrada:  torch.Size([8, 4])


Ahora  usemos  la  capa  de  incrustación  para  incrustar  estos  identificadores  de  token  en  un  modelo  de  256  dimensiones de vectores

In [10]:
tokens_embeddings = embedding_layer(inputs)
print(tokens_embeddings.shape)

torch.Size([8, 4, 256])


Para  un  enfoque  de  incrustación  absoluta  de  un  modelo  GPT,  solo  necesitamos  crear  otroa capa  de  incrustación  que  tiene  la  misma  dimensión  que  token_embedding_layer

In [11]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, salida_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)


torch.Size([4, 256])


Como se puede ver,  el  tensor  de  incrustación  posicional  consta  de  cuatro  vectores  de  256  dimensiones.
Ahora  podemos  agregarlos  directamente  a  las  incrustaciones  de  tokens,  donde  PyTorch  agregará  el  tensor  pos_embeddings  de  4x256  dimensiones  a  cada  tensor  de  incrustación  de  tokens  de  4x256  dimensiones  en  cada  uno  de  los  8  lotes

In [12]:
input_embeddings = tokens_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])


![Texto alternativo](./imgs/2.17.png)

Como  parte  del  proceso  de  procesamiento  de  entrada,  el  texto  de  entrada  se  divide  primero  en  tokens  individuales.  Estos  tokens  se  convierten  en  identificadores  de  token  mediante  un  vocabulario.  Los  identificadores  de  token  se  convierten  en  vectores  de  incrustación  a  los  que  se  añaden  incrustaciones  posicionales  de  tamaño  similar,  lo  que  da  como  resultado  incrustaciones  de  entrada  que  se  utilizan  como  entrada  para  las  capas  principales  de  LLM.